### Setup for this download file
- implement logging (should be able to open logging file while program is running)
    - timestamp, image, success?
- not time.sleep, but just do it every half second
- import random

In [2]:
import pandas as pd
import time

path_to_processed = '../data/processed/'


In [3]:
verbatim_file = path_to_processed + 'single_csv/verbatim.csv'

verbatim = pd.read_csv(verbatim_file, low_memory=False)
verbatim_df = pd.DataFrame(verbatim)
verbatim_df.head(3)

,gbifID,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,scientificName,higherClassification
0,4888478881,2017-10-16,Spain,Huesca,42.141713,0.362053,Petrophora convergata,Animalia|Geometridae
1,4888478983,2024-04-08,Germany,Nordrhein-Westfalen,50.798703,6.139380,Aglais io,Animalia|Nymphalidae
2,2844302871,2018-06-17,Germany,Rheinland-Pfalz,49.820400,6.920900,Pyrausta despicata,Animalia|Crambidae


In [4]:
multimedia_file = path_to_processed + 'single_csv/multimedia.csv'

multimedia = pd.read_csv(multimedia_file, low_memory=False)
multimedia_df = pd.DataFrame(multimedia)
multimedia_df.head(3)

,gbifID,format,identifier
0,1950935083,image/jpeg,https://observation.org/photos/8126181.jpg
1,1950946982,image/jpeg,https://observation.org/photos/8125216.jpg
2,1950947337,image/jpeg,https://observation.org/photos/8125217.jpg


In [5]:
species_counts = verbatim_df['scientificName'].value_counts().sort_values(ascending=False)
species_counts.head(5)

scientificName
Pararge aegeria     75939
Vanessa atalanta    61570
Noctua pronuba      50457
Aglais io           49595
Lycaena phlaeas     47993
Name: count, dtype: int64

In [6]:
print(f'Amount of classes with more than 1000 occurrences: {len(species_counts[species_counts > 1000])}')

Amount of classes with more than 1000 occurrences: 589


In [7]:
top589species = species_counts.head(589).keys()
print(len(top589species))

589


In [14]:
# Die anzahl der Samples zu downloaden wenn man die Anzahl auf 5000 pro spezies beschränkt
species_counts = species_counts.head(589)
large_species_samples = len(species_counts[species_counts > 5000]) * 5000
print(large_species_samples)
small_species_samples = species_counts[species_counts < 5000].sum()
print(small_species_samples)
print(f"Amount of samples to download with " \
            f"max 5000 images per species: {large_species_samples + small_species_samples}")

830000
979956
Amount of samples to download with max 5000 images per species: 1809956


In [ ]:
merged_df = pd.merge(verbatim[['gbifID', 'scientificName']], multimedia[['gbifID', 'identifier']], on='gbifID')
merged_df.head(3)

,gbifID,scientificName,identifier
0,4888478881,Petrophora convergata,https://observation.org/photos/87123955.jpg
1,4888478983,Aglais io,https://observation.org/photos/87147547.jpg
2,4888478983,Aglais io,https://observation.org/photos/87147548.jpg


In [ ]:
merged_df = merged_df.drop_duplicates(subset=['gbifID']) # reduce to single occ per gbifID
merged_df.head(3)

,gbifID,scientificName,identifier
0,4888478881,Petrophora convergata,https://observation.org/photos/87123955.jpg
1,4888478983,Aglais io,https://observation.org/photos/87147547.jpg
4,2844302871,Pyrausta despicata,https://observation.org/photos/17332900.jpg


In [ ]:
# Group by 'scientificName'
grouped = merged_df.groupby('scientificName')

# Initialize a list to hold the filtered DataFrames
filtered_dfs = []

for name, group in grouped:
    sample_size = len(group)
    if sample_size >= 5000:
        sampled = group.sample(n=5000, random_state=42) # classes with more than 5000 samples get reduced to 5000
        filtered_dfs.append(sampled)
    elif 1000 <= sample_size < 5000: # classes between 1000 and 5000 stay unchanged
        filtered_dfs.append(group)
    # Classes with less than 1000 samples are skipped

# Concatenate the filtered DataFrames
final_df = pd.concat(filtered_dfs)
print(final_df.shape)
final_df['scientificName'].value_counts()

(1809956, 3)


scientificName
Udea ferrugalis            5000
Tyria jacobaeae            5000
Zygaena filipendulae       5000
Triodia sylvina            5000
Tortrix viridana           5000
                           ... 
Parornix spec.             1006
Hypomecis roboraria        1004
Eupithecia tripunctaria    1004
Glaucopsyche alexis        1002
Cerastis rubricosa         1001
Name: count, Length: 589, dtype: int64

In [ ]:
# save dataset to file
# final_df.to_csv(path_to_processed + 'dataset_top589_max5000.csv', index=False)